In [1]:
from new_model import * 

In [2]:
seed = 100
random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
#Data 다운로드 경로지정
batch_size = 128


#Data Process 
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

train_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR10) * 95 / 100)
num_val = len(train_CIFAR10) - num_train
train_CIFAR10, val_CIFAR10 = torch.utils.data.random_split(train_CIFAR10, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR10, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#CIFAR 100 : Data 다운로드 경로지정
batch_size = 128

#Data Process
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

train_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR100) * 95 / 100)
num_val = len(train_CIFAR100) - num_train
train_CIFAR100, val_CIFAR100 = torch.utils.data.random_split(train_CIFAR100, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR100, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def node_prune(model, pruning_ratio):
    for layer in model.modules():
        if isinstance(layer, M_BasicBlock):
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[0][layer.M_relu1.active != 0]
            cutoff1 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu1.active.data[layer.get_2norm()[0] < cutoff1] *= 0
            
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[1][layer.M_relu2.active != 0]
            cutoff2 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu2.active.data[layer.get_2norm()[1] < cutoff2] *= 0

In [13]:
## fine_tuning
def fine_tuning(model, mode, EPOCHS, INITIAL_LR):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed100_CIFAR100"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)    
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

In [14]:
###8. test_model
def test_model(model):
    model.to(device)
    model.eval()

    total_examples = 0
    correct_examples = 0
    softmax = torch.nn.Softmax(dim=1)

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            pred = model(inputs)
            total_examples += inputs.shape[0]

            out = softmax(pred)
            out = torch.max(out, 1)

            correct_examples += torch.sum(targets==out[1]).cpu().data.numpy().tolist()

    avg_acc = correct_examples / total_examples
    print("Total examples is {}, correct examples is {}; Test accuracy: {}".format(total_examples, correct_examples, avg_acc))

## 모델지정

In [7]:
# # 모델 지정 및 GPU 지정
# device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
# model = Wide_ResNet(28, 10, 0.3, 100).to(device)
# check_prune = torch.load("../1.Wide_ResNet_model/saved_model/WR_28x10_cifar100.pth")
# model.load_state_dict(check_prune['state_dict'])
# # test_model(model)

| Wide-Resnet 28x10


<All keys matched successfully>

In [15]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

MOMENTUM = 0.9

REG = 5e-4

criterion = nn.CrossEntropyLoss()

ratio = 1-0.4**0.1976

basic = 10

In [16]:
for lr in [0.004, 0.02] :
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar100.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_0.6_multistepLR_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_0.6_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x10
최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:1'), tensor(3736., device='cuda:1'))
Epoch 10: progress time is 2692.09 sec
Validation loss: 0.0657, Validation accuracy: 0.9848
Epoch 20: progress time is 5381.93 sec
Validation loss: 0.0668, Validation accuracy: 0.9868
Epoch 30: progress time is 8072.05 sec
Validation loss: 0.0758, Validation accuracy: 0.9860
Epoch 40: progress time is 10840.65 sec
Validation loss: 0.0938, Validation accuracy: 0.9812
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:1'), tensor(3112., device='cuda:1'))
Epoch 50: progress time is 13732.45 sec
Validation loss: 0.3505, Validation accuracy: 0.9060
Epoch 60: progress time is 16532.09 sec
Validation loss: 0.3138, Validation accuracy: 0.9176
Epoch 70: progress time is 19606.39 sec
Validation loss: 0.2348, Validation accuracy: 0.9

# cosine scheduler

In [10]:
## fine_tuning
def fine_tuning(model, mode, EPOCHS, INITIAL_LR):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed100_CIFAR100"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)    
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

In [11]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar100.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_0.6_multistepLR_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_0.6_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x10
최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:1'), tensor(3736., device='cuda:1'))
Epoch 10: progress time is 2698.63 sec
Validation loss: 0.0576, Validation accuracy: 0.9884
Epoch 20: progress time is 5394.83 sec
Validation loss: 0.0648, Validation accuracy: 0.9860
Epoch 30: progress time is 8089.36 sec
Validation loss: 0.0876, Validation accuracy: 0.9812
Epoch 40: progress time is 10784.91 sec
Validation loss: 0.0842, Validation accuracy: 0.9848
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:1'), tensor(3112., device='cuda:1'))
Epoch 50: progress time is 13474.71 sec
Validation loss: 0.1820, Validation accuracy: 0.9576
Epoch 60: progress time is 16164.83 sec
Validation loss: 0.1647, Validation accuracy: 0.9652
Epoch 70: progress time is 18854.57 sec
Validation loss: 0.1596, Validation accuracy: 0.9

## 결과 확인

In [17]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_0.6_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_0.6_cosine_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x10
Total examples is 10000, correct examples is 7915; Test accuracy: 0.7915
| Wide-Resnet 28x10
Total examples is 10000, correct examples is 7827; Test accuracy: 0.7827
| Wide-Resnet 28x10
Total examples is 10000, correct examples is 7884; Test accuracy: 0.7884
| Wide-Resnet 28x10
Total examples is 10000, correct examples is 7920; Test accuracy: 0.792


In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = '28x10_CIFAR10_scheduler_cosine_lr0.1*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:1'), tensor(3736., device='cuda:1'))
Epoch 10: progress time is 2529.39 sec
Validation loss: 0.2105, Validation accuracy: 0.9340
Epoch 20: progress time is 5039.73 sec
Validation loss: 0.2226, Validation accuracy: 0.9332
Epoch 30: progress time is 7556.91 sec
Validation loss: 0.2452, Validation accuracy: 0.9288
Epoch 40: progress time is 10068.8 sec
Validation loss: 0.2201, Validation accuracy: 0.9400
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:1'), tensor(3112., device='cuda:1'))
Epoch 50: progress time is 12588.52 sec
Validation loss: 0.2263, Validation accuracy: 0.9372
Epoch 60: progress time is 15117.7 sec
Validation loss: 0.1511, Validation accuracy: 0.9580
Epoch 70: progress time is 17616.53 sec
Validation loss: 0.1407, Validation accuracy: 0.9600
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [12]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = '28x10_0.6_CIFAR10_scheduler_mult_lr0.1*0.2*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:0'), tensor(4480., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:0'), tensor(3736., device='cuda:0'))
Epoch 10: progress time is 2700.71 sec
Validation loss: 0.1402, Validation accuracy: 0.9612
Epoch 20: progress time is 5394.24 sec
Validation loss: 0.1591, Validation accuracy: 0.9548
Epoch 30: progress time is 8090.0 sec
Validation loss: 0.2017, Validation accuracy: 0.9412
Epoch 40: progress time is 11040.36 sec
Validation loss: 0.1533, Validation accuracy: 0.9560
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:0'), tensor(3112., device='cuda:0'))
Epoch 50: progress time is 13989.13 sec
Validation loss: 0.2245, Validation accuracy: 0.9368
Epoch 60: progress time is 17695.93 sec
Validation loss: 0.1893, Validation accuracy: 0.9456
Epoch 70: progress time is 20388.98 sec
Validation loss: 0.1389, Validation accuracy: 0.9580
Epoch 80: progr

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:1'), tensor(3736., device='cuda:1'))
Epoch 10: progress time is 2523.44 sec
Validation loss: 0.0625, Validation accuracy: 0.9876
Epoch 20: progress time is 5041.51 sec
Validation loss: 0.0617, Validation accuracy: 0.9884
Epoch 30: progress time is 7557.95 sec
Validation loss: 0.0771, Validation accuracy: 0.9832
Epoch 40: progress time is 10073.43 sec
Validation loss: 0.0901, Validation accuracy: 0.9832
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:1'), tensor(3112., device='cuda:1'))
Epoch 50: progress time is 12593.96 sec
Validation loss: 0.3467, Validation accuracy: 0.9004
Epoch 60: progress time is 15124.31 sec
Validation loss: 0.3305, Validation accuracy: 0.9104
Epoch 70: progress time is 17651.85 sec
Validation loss: 0.2623, Validation accuracy: 0.9352
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:1'), tensor(3736., device='cuda:1'))
Epoch 10: progress time is 2522.41 sec
Validation loss: 0.7077, Validation accuracy: 0.8040
Epoch 20: progress time is 5039.43 sec
Validation loss: 0.7873, Validation accuracy: 0.7928
Epoch 30: progress time is 7550.57 sec
Validation loss: 0.8077, Validation accuracy: 0.7788
Epoch 40: progress time is 10068.45 sec
Validation loss: 0.8540, Validation accuracy: 0.7704
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:1'), tensor(3112., device='cuda:1'))
Epoch 50: progress time is 12592.23 sec
Validation loss: 0.9083, Validation accuracy: 0.7564
Epoch 60: progress time is 15121.25 sec
Validation loss: 1.0312, Validation accuracy: 0.7432
Epoch 70: progress time is 17649.12 sec
Validation loss: 0.6681, Validation accuracy: 0.8232
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

## seed 10

In [15]:
##scheduler 실행 : multistepLR , lr 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:0'), tensor(4480., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:0'), tensor(3736., device='cuda:0'))
Epoch 10: progress time is 2717.24 sec
Validation loss: 0.0118, Validation accuracy: 0.9964
Epoch 20: progress time is 5427.68 sec
Validation loss: 0.0161, Validation accuracy: 0.9960
Epoch 30: progress time is 8138.01 sec
Validation loss: 0.0149, Validation accuracy: 0.9968
Epoch 40: progress time is 10849.08 sec
Validation loss: 0.0280, Validation accuracy: 0.9916
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:0'), tensor(3112., device='cuda:0'))
Epoch 50: progress time is 13550.98 sec
Validation loss: 0.1227, Validation accuracy: 0.9596
Epoch 60: progress time is 16253.19 sec
Validation loss: 0.0752, Validation accuracy: 0.9752
Epoch 70: progress time is 18955.02 sec
Validation loss: 0.0265, Validation accuracy: 0.9908
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##scheduler 실행 : multistepLR , lr 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:2'), tensor(4480., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:2'), tensor(3736., device='cuda:2'))
Epoch 10: progress time is 3364.86 sec
Validation loss: 0.1541, Validation accuracy: 0.9456
Epoch 20: progress time is 7386.3 sec
Validation loss: 0.1844, Validation accuracy: 0.9440
Epoch 30: progress time is 10234.53 sec
Validation loss: 0.1951, Validation accuracy: 0.9408
Epoch 40: progress time is 13181.36 sec
Validation loss: 0.1797, Validation accuracy: 0.9488
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:2'), tensor(3112., device='cuda:2'))
Epoch 50: progress time is 16816.63 sec
Validation loss: 0.1909, Validation accuracy: 0.9372
Epoch 60: progress time is 20430.65 sec
Validation loss: 0.2084, Validation accuracy: 0.9372
Epoch 70: progress time is 23280.47 sec
Validation loss: 0.1076, Validation accuracy: 0.9680
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##cifar100 
##scheduler 실행 : multistepLR , lr 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:0'), tensor(4480., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:0'), tensor(3736., device='cuda:0'))
Epoch 10: progress time is 3003.98 sec
Validation loss: 0.0912, Validation accuracy: 0.9820
Epoch 20: progress time is 6504.1 sec
Validation loss: 0.0811, Validation accuracy: 0.9848
Epoch 30: progress time is 9590.9 sec
Validation loss: 0.0809, Validation accuracy: 0.9860
Epoch 40: progress time is 12323.84 sec
Validation loss: 0.1062, Validation accuracy: 0.9772
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:0'), tensor(3112., device='cuda:0'))
Epoch 50: progress time is 15056.59 sec
Validation loss: 0.3893, Validation accuracy: 0.8988
Epoch 60: progress time is 17796.11 sec
Validation loss: 0.3513, Validation accuracy: 0.9072
Epoch 70: progress time is 20537.48 sec
Validation loss: 0.2733, Validation accuracy: 0.9256
Epoch 80: progress

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##cifar100 
##scheduler 실행 : multistepLR , lr 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:2'), tensor(4480., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:2'), tensor(3736., device='cuda:2'))
Epoch 10: progress time is 2739.25 sec
Validation loss: 0.6960, Validation accuracy: 0.8056
Epoch 20: progress time is 5470.1 sec
Validation loss: 0.7759, Validation accuracy: 0.7916
Epoch 30: progress time is 8200.82 sec
Validation loss: 0.9642, Validation accuracy: 0.7568
Epoch 40: progress time is 10937.89 sec
Validation loss: 0.8706, Validation accuracy: 0.7684
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:2'), tensor(3112., device='cuda:2'))
Epoch 50: progress time is 13649.64 sec
Validation loss: 1.0774, Validation accuracy: 0.7200
Epoch 60: progress time is 16352.32 sec
Validation loss: 1.0680, Validation accuracy: 0.7248
Epoch 70: progress time is 19061.2 sec
Validation loss: 0.6628, Validation accuracy: 0.8324
Epoch 80: progress

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

## 결과 확인

In [10]:
## cifar10
check_prune = torch.load("./saved_tuning/28x10_CIFAR10_scheduler_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_CIFAR10_scheduler_cosine finished')

Total examples is 10000, correct examples is 9563; Test accuracy: 0.9563


In [13]:
## cifar10
check_prune = torch.load("./saved_tuning/28x10_0.6_CIFAR10_scheduler_mult_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_CIFAR10_scheduler_cosine finished')

Total examples is 10000, correct examples is 9582; Test accuracy: 0.9582


In [10]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x10_0.6_multstepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x10_0.6_multstepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7906; Test accuracy: 0.7906


In [10]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x10_0.6_multstepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x10_0.6_multstepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7883; Test accuracy: 0.7883


## seed 10

In [13]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x10_0.6_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9554; Test accuracy: 0.9554


In [14]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR10/28x10_0.6_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9570; Test accuracy: 0.957


In [ ]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR10/28x10_0.6_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')b

In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR10/28x10_0.6_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x10_0.6_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9554; Test accuracy: 0.9554


In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR10/28x10_0.6_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x10_0.6_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9568; Test accuracy: 0.9568


In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x10_0.6_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x10_0.6_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7866; Test accuracy: 0.7866


In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x10_0.6_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x10_0.6_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7851; Test accuracy: 0.7851
